In [16]:
import pandas as pd
import numpy as np
from apyori import apriori  

In [17]:
df = pd.read_csv("../data/location1_cleaned.csv")
df_prods = pd.read_csv("../data_engineering/products_charla20181127.csv")

In [18]:
del df['product_sku']
del df['display_description']
del df['location_name']
del df_prods['location']

In [19]:
df.head(10)

,order_id,order_time(UTC),items_id,items_name,item_price,quantity,served_for,item_category_type
0,841222,7/31/18 19:02,2255,Bud Light,7.50,1,Lunch,Alcoholic Beverage
1,841222,7/31/18 19:02,3036,Shrimp Platter,12.49,1,Lunch,Entree
2,841222,7/31/18 19:02,3038,"Shrimp, Fish, & Chicken Platter",15.99,1,Lunch,Entree
3,841222,7/31/18 19:02,3038,"Shrimp, Fish, & Chicken Platter",15.99,1,Lunch,Entree
4,841222,7/31/18 19:02,3036,Shrimp Platter,12.49,1,Lunch,Entree
5,841222,7/31/18 19:02,1858,Fountain Beverage,3.29,1,Lunch,Beverage
6,841231,7/31/18 19:05,3236,,4.99,1,Lunch,Dessert
7,841231,7/31/18 19:05,3236,,4.99,1,Lunch,Dessert
8,841237,7/31/18 19:08,2716,Chicken Strips,9.49,1,Lunch,Entree
9,841237,7/31/18 19:08,1858,Fountain Beverage,3.69,1,Lunch,Beverage


In [20]:
# make large and small fountain drinks be the same thing
#df.loc[ df['items_id'] == 2253,'items_id' ] = 2252 
# combine the same cupcake
#df.loc[ df['items_id'] == 3411] = 3236
# combine the same Olaf cupcake
#df.loc[ df['items_id'] == 3410] = 2245
# combine mac and chees
#df.loc[ df['items_id'] == 3274] = 3276

In [21]:
#Categories:
cats = list(set(list(df_prods['category'])))
cats


['Side', 'Dessert', 'Beverage', 'Entree', 'Kids', 'Alcoholic Beverage']

In [23]:
# most popular items
ydf = (df.groupby('items_id').count()['order_id']).reset_index(drop=False)
ydf.rename(columns={'order_id':'count'},inplace=True)
df_prods.rename(columns={'id':'items_id'},inplace=True)
zdf = pd.merge(ydf,df_prods,on='items_id',how='right')
zdf = zdf.sort_values(['count'],ascending=False)
zdf.head(5)

,items_id,count,name,display_desc,category,img,name_short,category_corrected
8,1858,13730.0,Assorted Fountain Beverages - Regular,Assorted Coca-Cola® Offerings,Beverage,bev_fountain_beverages.jpeg,Fountain Beverage,NaN
17,2237,5542.0,Mediterranean Salad with Chicken,tossed in a Mediterranean Vinaigrette and serv...,Entree,salad_mediterranean_salad_chicken.png,Chicken Mediterranean Salad,NaN
29,2716,4894.0,Chicken Breast Strips,"served with your choice of fresh Green Beans, ...",Entree,fried_chicken-strips.png,Chicken Strips,NaN
33,3038,4764.0,"Shrimp, Fish and Chicken Platter",served with Steak Fries and Coleslaw,Entree,fried_shrimp_fish_chicken.png,"Shrimp, Fish, & Chicken Platter",NaN
11,2210,4548.0,1/3 lb Angus Bacon Cheeseburger,"served with your choice of fresh Green Beans, ...",Entree,burger_angus_bacon_cheeseburger.jpeg,NaN,NaN


In [24]:
entrees = zdf[ zdf['category'] == "Entree" ]

In [25]:
bev = zdf[ zdf['category'] == "Beverage" ]

In [26]:
alc = zdf[ zdf['category'] == "Alcoholic Beverage" ]

In [27]:
sides = zdf[ zdf['category'] == "Side" ]

In [28]:
desserts = zdf[ zdf['category'] == "Dessert" ]

In [29]:
kids = zdf[ zdf['category'] == "Kids" ]

In [30]:
import collections
most_popular = collections.OrderedDict()

In [31]:
most_popular['entree'] = list(entrees.iloc[0:3][ 'items_id' ])
most_popular['side'] = list(sides.iloc[0:3][ 'items_id' ])
most_popular['dessert'] = list(desserts.iloc[0:3][ 'items_id' ])
most_popular['beverage'] = list(bev.iloc[0:3][ 'items_id' ])
most_popular['alcohol'] = list(alc.iloc[0:3]['items_id'])
most_popular['kids'] = list(kids.iloc[0:3]['items_id'])

In [32]:
import json
with open("../data/most_popular.json","w") as outfile:
    outfile.write(json.dumps(most_popular))